## India Covid APP

In [1]:
import json
import os.path
import re
import urllib.request
from datetime import date, datetime, timedelta

import dash
import dash_bootstrap_components as dbc
import dash_core_components as dcc
import dash_html_components as html
import dash_table
import numpy as np
from flask import Markup
from plotly.offline import plot
import pandas as pd
import plotly
import plotly.express as px
import plotly.graph_objects as go
import requests
from bs4 import BeautifulSoup
from flask import Flask, render_template
from flask import Markup

world_df = None
df_nation = None

daily_confirmed = []
daily_recovered = []
daily_deceased = []
daily_active = []
daily_tested = []

total_confirmed = []
total_recovered = []
total_deceased = []
total_active = []
total_tested = []

dates = []

app = Flask(__name__)

def prepare_world_data():
	global world_df
	data_date = (date.today() - timedelta(days=2)).strftime('%m-%d-%Y')
	world_df = pd.read_csv(
		'https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_daily_reports/{}.csv'.format(
			data_date))
	world_df = world_df[['Country_Region', 'Confirmed', 'Deaths', 'Recovered', 'Active']]
	world_df = world_df.groupby('Country_Region').sum()
	world_df = world_df.reset_index()

def world_heatmap():
	prepare_world_data()

	object = {
		'countries' : world_df['Country_Region'],
		'confirmed' : world_df['Confirmed']
	}

	graphJSON = json.dumps(object, cls=plotly.utils.PlotlyJSONEncoder)
	return graphJSON

def toint(x):
	if (x == ''):
		return None
	elif (type(x) == 'str'):
		return int(x)
	else:
		return x


def checkEmptyRate(x):
	if (x == ''):
		return None
	elif (type(x) == 'int'):
		return str(x)
	else:
		return x


def preprocess(x):
	if (type(x) == 'str'):
		return int(x.replace(',', ''))
	return x


def get_numbers():
	global df_nation

	global dates

	global daily_confirmed
	global daily_recovered
	global daily_deceased
	global daily_active
	global daily_tested

	global total_confirmed
	global total_recovered
	global total_deceased
	global total_active
	global total_tested

	URL = 'https://api.covid19india.org/data.json'
	POP = 1352600000 / 1000000

	with urllib.request.urlopen(URL) as url:
		data = json.loads(url.read().decode())

	for d in range(len(data['cases_time_series'])):
		date_string = data['cases_time_series'][d]['date']
		try:
			date = datetime.strptime(date_string, '%d %B ')
		except:
			date = datetime.strptime('29 February 2020 ', '%d %B %Y ')
		date = date.strftime('%d/%m/2020')
		dates.append(date)

	daily_confirmed = [int(i.get('dailyconfirmed', None)) for i in data.get('cases_time_series', None)]
	daily_deceased = [int(i.get('dailydeceased', None)) for i in data.get('cases_time_series', None)]
	daily_recovered = [int(i.get('dailyrecovered', None)) for i in data.get('cases_time_series', None)]

	total_confirmed = [int(i.get('totalconfirmed', None)) for i in data.get('cases_time_series', None)]
	total_deceased = [int(i.get('totaldeceased', None)) for i in data.get('cases_time_series', None)]
	total_recovered = [int(i.get('totalrecovered', None)) for i in data.get('cases_time_series', None)]

	offset = 40

	daily_tested = [0 for i in range(offset)] + [toint(i.get('samplereportedtoday', None)) for i in
												 data.get('tested', None)]
	positivity_rate = [0 for i in range(offset)] + [checkEmptyRate(i.get('testpositivityrate', None)) for i in
													data.get('tested', None)]
	total_tested = [0 for i in range(offset)] + [toint(i.get('totalsamplestested', None)) for i in
												 data.get('tested', None)]

	df_nation = pd.DataFrame()
	df_nation['Daily Confirmed'] = daily_confirmed
	df_nation['Daily Deceased'] = daily_deceased
	df_nation['Daily Recovered'] = daily_recovered
	df_nation['Daily Active'] = df_nation['Daily Confirmed'] - df_nation['Daily Deceased'] - df_nation[
		'Daily Recovered']
	df_nation['Total Confirmed'] = total_confirmed
	df_nation['Total Deceased'] = total_deceased
	df_nation['Total Recovered'] = total_recovered
	df_nation['Total Active'] = df_nation['Total Confirmed'] - df_nation['Total Deceased'] - df_nation[
		'Total Recovered']
	df_nation['Daily Tested'] = daily_tested
	df_nation['Total Tested'] = total_tested
	df_nation['Cases PM'] = np.round(np.array(total_confirmed).astype(float) / POP)
	df_nation['Tests PM'] = np.round(np.array(total_tested).astype(float) / POP)
	df_nation['Positivity Rate'] = positivity_rate
	df_nation.index = dates

	for c in range(0, len(df_nation.columns) - 3):
		df_nation[df_nation.columns.values[c]].map(preprocess)

	df_nation = df_nation.fillna(method='ffill')
	df_nation = df_nation.T

	dates.reverse()
	df_nation = df_nation[dates]

	sign = lambda x: '+' + str(x) if (x >= 0) else str(x)

	object = {'cum_numbers': [total_confirmed[-1],total_recovered[-1],total_deceased[-1],
						  str(int(total_confirmed[-1]) - ( int(total_deceased[-1]) + int(total_recovered[-1]) )),
						  total_tested[-1]],
			  'daily_numbers': ['+'+str(daily_confirmed[-1]),'+'+str(daily_recovered[-1]), '+'+str(daily_deceased[-1]),
						   str(sign(int(daily_confirmed[-1]) - ( int(daily_deceased[-1]) + int(daily_recovered[-1]) ))),
						  '+'+str(daily_tested[-1])] }

	graphJSON = json.dumps(object, cls=plotly.utils.PlotlyJSONEncoder)

	return graphJSON


@app.route('/')
def index():
	world_heat = world_heatmap()
	numbers = get_numbers()

	npie_labels = ['Confirmed', 'Deceased', 'Recovered', 'Active']
	npie_cases = [df_nation.loc['Total Confirmed', dates[-1]], df_nation.loc['Total Deceased', dates[-1]],
			 df_nation.loc['Total Recovered', dates[-1]], df_nation.loc['Total Active', dates[-1]]]

	return render_template('index.html', world_heat=world_heat, numbers = numbers, world_cols = world_df.columns.values, world_rows = world_df.values.tolist(),
						   nation_cols = df_nation.reset_index().columns.values, nation_rows = df_nation.reset_index().values.tolist(), npie_labels = npie_labels,
						   npie_cases = npie_cases
						   )



In [5]:

def getStateName(code):
	state = {'CT': 'Chhattisgarh',
             'TG' : 'Telangana',
             'TN' : 'Tamil Nadu',
             'LA' : 'Ladakh',
             'DN' : 'Dadra and Nagar Haveli',
             'UT' : 'Uttarakhand',
             'TT' : 'Total' }
	return state.get(code, None)


try:
    fh = open('data.json', 'r')
    if (len(fh.read()) > 0):
        state_names = json.loads(fh.read())
except:
    URL = 'https://api.covid19india.org/v4/data.json'
    URL_STATE_ABR = 'https://slusi.dacnet.nic.in/watershedatlas/list_of_state_abbreviation.htm'

    with urllib.request.urlopen(URL) as url:
        data = json.loads(url.read().decode())

    code_page = requests.get(URL_STATE_ABR)
    soup = BeautifulSoup(code_page.content, 'html.parser')

    keys = []
    values = []
    raw_list = []

    page_code = soup.find_all('span')

    for i in page_code:
        raw_list.append(re.findall('.*HI.>([A-Z].*)<o:p></o:p>.*', i.decode()))

    headers = ['Sl. No.', 'State/ UT', 'Abbreviation']

    for d in raw_list:
        if (len(d) > 0 and d[0] not in headers):
            if (len(d[0]) > 2):
                values.append(d[0])
            else:
                keys.append(d[0])

    state_names = dict(list(zip(keys, values)))

    with open('data.json', 'w') as fp:
        json.dump(state_names, fp)


states = []
for state in data.keys():
    states.append(state)

total = []
for state in states:
    try:
        cases = data[state]['total']
    except:
        cases = None
    total.append(cases)

confirmed = []
deceased = []
recovered = []
tested = []

for cases in total:
    confirmed.append(cases.get('confirmed', -1))
    deceased.append(cases.get('deceased', -1))
    recovered.append(cases.get('recovered', -1))
    tested.append(cases.get('tested', -1))

df_state = pd.DataFrame()
df_state['Code'] = states
df_state['State'] = [state_names.get(i, getStateName(i)) for i in states]
df_state['Confirmed'] = confirmed
df_state['Deceased'] = deceased
df_state['Recovered'] = recovered
df_state['Active'] = df_state['Confirmed'] - df_state['Recovered'] - df_state['Deceased']
df_state['Tested'] = tested

df_state = df_state.sort_values(by=['Confirmed'], ascending=False)
df_state = df_state.reset_index(drop=True)
df_state = df_state[1:]

In [12]:
df_state[df_state['State']=='Maharashtra']['Confirmed'].values[0]

1351153

In [56]:
array = np.array(total_deceased)
array[array==-inf] = 1
array[array==0] = 1
np.log(array)


array([ 0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.69314718,
        0.69314718,  0.69314718,  1.09861229,  1.09861229,  1.38629436,
        1.38629436,  1.38629436,  1.94591015,  2.19722458,  2.30258509,
        2.39789527,  2.77258872,  2.94443898,  3.17805383,  3.29583687,
        3.71357207,  3.8501476 ,  3.97029191,  4.2341065 ,  4.41884061,
        4.56434819,  4.77068462,  4.8978398 ,  5.08140436,  5.19

In [49]:
np.log(array)

<ipython-input-49-aa768b74dbc5>:1: RuntimeWarning:

divide by zero encountered in log



array([       -inf,        -inf,        -inf,        -inf,        -inf,
              -inf,        -inf,        -inf,        -inf,        -inf,
              -inf,        -inf,        -inf,        -inf,        -inf,
              -inf,        -inf,        -inf,        -inf,        -inf,
              -inf,        -inf,        -inf,        -inf,        -inf,
              -inf,        -inf,        -inf,        -inf,        -inf,
              -inf,        -inf,        -inf,        -inf,        -inf,
              -inf,        -inf,        -inf,        -inf,        -inf,
              -inf,        -inf,  0.        ,  0.        ,  0.69314718,
        0.69314718,  0.69314718,  1.09861229,  1.09861229,  1.38629436,
        1.38629436,  1.38629436,  1.94591015,  2.19722458,  2.30258509,
        2.39789527,  2.77258872,  2.94443898,  3.17805383,  3.29583687,
        3.71357207,  3.8501476 ,  3.97029191,  4.2341065 ,  4.41884061,
        4.56434819,  4.77068462,  4.8978398 ,  5.08140436,  5.19

In [56]:
df_nation.loc['Positivity Rate',:].str.replace('%','').replace('',0).fillna(method='ffill').replace(method='ffill').astype(float).tolist()


[8.62,
 8.65,
 8.64,
 8.63,
 8.62,
 8.64,
 8.62,
 8.61,
 8.61,
 8.61,
 8.61,
 8.61,
 8.61,
 8.61,
 8.61,
 8.62,
 8.62,
 8.63,
 8.61,
 8.61,
 8.61,
 8.43,
 8.46,
 8.49,
 8.51,
 8.55,
 8.54,
 8.57,
 8.57,
 8.57,
 8.58,
 8.59,
 8.65,
 8.62,
 8.62,
 8.68,
 8.68,
 8.72,
 8.73,
 8.81,
 8.83,
 8.84,
 8.88,
 8.92,
 8.95,
 8.97,
 9.01,
 8.93,
 8.92,
 8.89,
 8.86,
 8.87,
 8.89,
 8.93,
 8.84,
 8.77,
 8.71,
 8.71,
 8.63,
 8.55,
 8.54,
 8.51,
 8.44,
 8.35,
 8.22,
 8.11,
 8.03,
 7.96,
 7.82,
 7.75,
 7.69,
 7.62,
 7.55,
 7.51,
 7.45,
 7.34,
 7.28,
 7.21,
 7.16,
 7.1,
 7.05,
 7.0,
 6.88,
 6.81,
 6.75,
 6.68,
 6.64,
 6.59,
 6.54,
 6.44,
 6.37,
 6.32,
 6.26,
 6.2,
 6.17,
 6.14,
 6.05,
 5.98,
 5.93,
 5.88,
 5.82,
 5.79,
 5.77,
 5.68,
 5.62,
 5.56,
 5.51,
 5.45,
 5.41,
 5.39,
 5.28,
 5.22,
 5.17,
 5.11,
 5.05,
 5.0,
 4.97,
 4.87,
 4.8,
 4.75,
 4.7,
 4.65,
 4.64,
 4.57,
 4.47,
 4.4,
 4.35,
 4.29,
 4.24,
 4.17,
 4.16,
 4.07,
 4.02,
 4.02,
 4.01,
 4.01,
 4.02,
 4.01,
 3.91,
 3.92,
 3.92,
 3.91,
 3.87,
 3.9,
